# Обогащение данных

1. Заполнить отсутствующие значения по погоде интерполяционными данными.
2. Для точки росы вычесть температуру воздуха.
3. Напрвыление ветра разложить на синус и косинус.
4. Для тем. воздуха вычеслить 1-ю и 2-ю производные.
5. Вывести параметры по праздничным дням, дням недели, месяцам и неделям года.
6. Посчитать модель линейной регрессии по первым 20 зданиям и найти точность.

In [29]:
import pandas as pd
import numpy as np

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from core.reduce_mem_usage import reduce_mem_usage

## Загрузка данных

In [2]:
buildings = pd.read_csv("../data/buildings.csv")
weather = pd.read_csv("../data/weather.csv")
energy = pd.read_csv("../data/train.0.csv")

## Отобрать 20 зданий и объединить

In [7]:
energy = energy[energy["building_id"] < 20]
energy = pd.merge(
    left=energy,
    right=buildings,
    how="left",
    left_on="building_id",
    right_on="building_id",
)
del buildings
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   building_id    175680 non-null  int64  
 1   meter          175680 non-null  int64  
 2   timestamp      175680 non-null  object 
 3   meter_reading  175680 non-null  float64
 4   site_id        175680 non-null  int64  
 5   primary_use    175680 non-null  object 
 6   square_feet    175680 non-null  int64  
 7   year_built     175680 non-null  float64
 8   floor_count    0 non-null       float64
dtypes: float64(3), int64(4), object(2)
memory usage: 13.4+ MB
None


## Интерполяция значений для weather

In [9]:
weather = weather[weather["site_id"] == 0]
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x: x if x > 0 else 0)
interpolate_columns = [
    "air_temperature",
    "dew_temperature",
    "cloud_coverage",
    "wind_speed",
    "wind_direction",
    "precip_depth_1_hr",
    "sea_level_pressure",
]

for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction="both", kind="cubic")

## Обогащение данных: погода

In [11]:
weather["wind_direction_rad"] = weather["wind_direction"] * np.pi/180
weather["wind_direction_sin"] = np.sin(weather["wind_direction_rad"])
weather["wind_direction_cos"] = np.cos(weather["wind_direction_rad"])
weather["air_temperature_diff_1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff_1"] = weather.at[1, "air_temperature_diff_1"]
weather["air_temperature_diff_2"] = weather["air_temperature_diff_1"].diff()
weather.at[0, "air_temperature_diff_2"] = weather.at[1, "air_temperature_diff_2"]

## Объединить погодные данные

In [30]:
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(
    left=energy,
    right=weather,
    how="left",
    left_index=True,
    right_index=True,
)
energy.reset_index(inplace=True)
energy = energy.drop(
    columns=[
        "meter",
        "site_id",
        "year_built",
        "square_feet",
        "floor_count",
    ], 
    axis=1
)
del weather
energy = reduce_mem_usage(energy)
print(energy.info())

Потребление памяти меньше на - 15.41 Мб (минус 71.9%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               175680 non-null  datetime64[ns]
 1   building_id             175680 non-null  int8          
 2   meter_reading           175680 non-null  float16       
 3   primary_use             175680 non-null  category      
 4   air_temperature         175680 non-null  float16       
 5   cloud_coverage          175680 non-null  float16       
 6   dew_temperature         175680 non-null  float16       
 7   precip_depth_1_hr       175680 non-null  float16       
 8   sea_level_pressure      175680 non-null  float16       
 9   wind_direction          175680 non-null  float16       
 10  wind_speed              175680 non-null  float16       
 11  wind_direction_rad      175680 non-nu

## Обогащение данных: дата

In [32]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.isocalendar().week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)

dates_range = pd.date_range(start="2015-12-31", end="2017-01-01")
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
energy["is_holiday"] = energy["date"].isin(us_holidays).astype("int8")

for weekday in range(0, 7):
    energy[f"is_wday {str(weekday)}"] = energy["weekday"].isin([weekday]).astype("int8")
for week in range(1, 54):
    energy[f"is_w {str(week)}"] = energy["week"].isin([week]).astype("int8")
for month in range(1, 13):
    energy[f"is_m {str(month)}"] = energy["month"].isin([month]).astype("int8")

## Логарифмирование данных

In [42]:
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

## Разделение данных

In [43]:
energy_train, energy_test = train_test_split(
    energy[energy["meter_reading"] > 0],
    test_size=0.2,
)
print(energy_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86858 entries, 69792 to 106572
Data columns (total 95 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   timestamp               86858 non-null  datetime64[ns]
 1   building_id             86858 non-null  int8          
 2   meter_reading           86858 non-null  float16       
 3   primary_use             86858 non-null  category      
 4   air_temperature         86858 non-null  float16       
 5   cloud_coverage          86858 non-null  float16       
 6   dew_temperature         86858 non-null  float16       
 7   precip_depth_1_hr       86858 non-null  float16       
 8   sea_level_pressure      86858 non-null  float16       
 9   wind_direction          86858 non-null  float16       
 10  wind_speed              86858 non-null  float16       
 11  wind_direction_rad      86858 non-null  float16       
 12  wind_direction_sin      86858 non-null  f